In [ ]:
import pandas as pd


In [ ]:
df=pd.read_csv("data.csv")

In [ ]:
df.head(10)


,Comment,Emotion
0,i seriously hate one subject to death but now ...,fear
1,im so full of life i feel appalled,anger
2,i sit here to write i start to dig out my feel...,fear
3,ive been really angry with r and i feel like a...,joy
4,i feel suspicious if there is no one outside l...,fear
5,i feel jealous becasue i wanted that kind of l...,anger
6,when a friend of mine keeps telling me morbid ...,anger
7,i finally fell asleep feeling angry useless an...,anger
8,i feel a bit annoyed and antsy in a good way,anger
9,i feel like i ve regained another vital part o...,joy


In [ ]:
df.shape

(5937, 2)

In [ ]:
#adding a new column Emotion_num which gives unidue number to these emotions
df["Emotion_num"]=df.Emotion.map({
    'joy':0,
    'fear':1,
    'anger':2
})
df.head(10)

,Comment,Emotion,Emotion_num
0,i seriously hate one subject to death but now ...,fear,1
1,im so full of life i feel appalled,anger,2
2,i sit here to write i start to dig out my feel...,fear,1
3,ive been really angry with r and i feel like a...,joy,0
4,i feel suspicious if there is no one outside l...,fear,1
5,i feel jealous becasue i wanted that kind of l...,anger,2
6,when a friend of mine keeps telling me morbid ...,anger,2
7,i finally fell asleep feeling angry useless an...,anger,2
8,i feel a bit annoyed and antsy in a good way,anger,2
9,i feel like i ve regained another vital part o...,joy,0


Modellind without preprocessing text data


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.Comment,df.Emotion_num,test_size=0.2,random_state=2022,stratify=df.Emotion_num)

In [ ]:
x_train.shape

(4749,)

In [ ]:
x_test.shape

(1188,)

Using CountVectorizer with only trigrams and RandomForest as the classifier.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
clf=Pipeline([
    ('vectorizer_bow',CountVectorizer(ngram_range=(3,3))),
    ('classifier',RandomForestClassifier())
])

In [ ]:
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(3, 3))),
                ('classifier', RandomForestClassifier())])

In [ ]:
y_pred=clf.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.59      0.25      0.35       400
           1       0.36      0.81      0.50       388
           2       0.53      0.20      0.30       400

    accuracy                           0.42      1188
   macro avg       0.50      0.42      0.38      1188
weighted avg       0.50      0.42      0.38      1188



Using CountVectorizer with both unigram and bigrams and  Multinomial Naive Bayes as the classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf=Pipeline([
    ('vectorizer_bow',CountVectorizer(ngram_range=(1,2))),
    ('nb',MultinomialNB())
])

In [ ]:
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(1, 2))),
                ('nb', MultinomialNB())])

In [ ]:
y_pred=clf.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.86      0.87       400
           1       0.87      0.83      0.85       388
           2       0.83      0.88      0.85       400

    accuracy                           0.86      1188
   macro avg       0.86      0.86      0.86      1188
weighted avg       0.86      0.86      0.86      1188



Using CountVectorizer with both unigram and Bigrams and RandomForest as the classifier.

In [ ]:
clf=Pipeline([
    ('vectorizer_bow',CountVectorizer(ngram_range=(1,2))),
    ('classifier',RandomForestClassifier())
])

In [ ]:
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(1, 2))),
                ('classifier', RandomForestClassifier())])

In [ ]:
y_pred=clf.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.85      0.97      0.90       400
           1       0.94      0.88      0.91       388
           2       0.93      0.86      0.89       400

    accuracy                           0.90      1188
   macro avg       0.91      0.90      0.90      1188
weighted avg       0.91      0.90      0.90      1188



Using TF-IDF vectorizer for Pre-processing the text and RandomForest as the classifier.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
clf=Pipeline([
    ('vectorizer_tfidf',TfidfVectorizer()),
    ('classifier',RandomForestClassifier())
])

In [ ]:
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('classifier', RandomForestClassifier())])

In [ ]:
y_pred=clf.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.95      0.91       400
           1       0.92      0.89      0.90       388
           2       0.93      0.86      0.90       400

    accuracy                           0.90      1188
   macro avg       0.91      0.90      0.90      1188
weighted avg       0.91      0.90      0.90      1188



Using text pre-processing to remove stop words, punctuations and apply lemmatization

In [ ]:
import spacy

# load english language model and create nlp object from it
nlp = spacy.load("en_core_web_sm") 


#use this utility function to get the preprocessed text data
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens) 

In [ ]:
df['preprocessed_text']=df['Comment'].apply(preprocess)

In [ ]:
df.head(10)

,Comment,Emotion,Emotion_num,preprocessed_text
0,i seriously hate one subject to death but now ...,fear,1,seriously hate subject death feel reluctant drop
1,im so full of life i feel appalled,anger,2,m life feel appalled
2,i sit here to write i start to dig out my feel...,fear,1,sit write start dig feeling think afraid accep...
3,ive been really angry with r and i feel like a...,joy,0,ve angry r feel like idiot trust place
4,i feel suspicious if there is no one outside l...,fear,1,feel suspicious outside like rapture happen
5,i feel jealous becasue i wanted that kind of l...,anger,2,feel jealous becasue want kind love true conne...
6,when a friend of mine keeps telling me morbid ...,anger,2,friend keep tell morbid thing happen dog
7,i finally fell asleep feeling angry useless an...,anger,2,finally fall asleep feel angry useless anxiety
8,i feel a bit annoyed and antsy in a good way,anger,2,feel bit annoyed antsy good way
9,i feel like i ve regained another vital part o...,joy,0,feel like ve regain vital life live


Building a model with pre processed text

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.preprocessed_text,df.Emotion_num,test_size=0.2,random_state=2022,stratify=df.Emotion_num)

using CountVectorizer with both unigrams and bigrams and RandomForest as the classifier

In [ ]:
clf=Pipeline([
    ('vectorizer_bow',CountVectorizer(ngram_range=(1,2))),
    ('classifier',RandomForestClassifier())
])

In [ ]:
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer_bow', CountVectorizer(ngram_range=(1, 2))),
                ('classifier', RandomForestClassifier())])

In [ ]:
y_pred=clf.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95       400
           1       0.94      0.92      0.93       388
           2       0.92      0.94      0.93       400

    accuracy                           0.93      1188
   macro avg       0.93      0.93      0.93      1188
weighted avg       0.93      0.93      0.93      1188



Using TF-IDF vectorizer for pre-processing the text and RandomForest as the classifier.

In [ ]:
clf=Pipeline([
    ('vectorizer_tfidf',TfidfVectorizer()),
    ('classifier',RandomForestClassifier())
])

In [ ]:
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('classifier', RandomForestClassifier())])

In [ ]:
y_pred=clf.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94       400
           1       0.93      0.92      0.92       388
           2       0.94      0.91      0.92       400

    accuracy                           0.93      1188
   macro avg       0.93      0.93      0.93      1188
weighted avg       0.93      0.93      0.93      1188



**Key Findings:**

* As the n_gram range keeps increasing, there's drastic fall of improvement in performance metrics.

* There's seen a significant improvement in results before pre-processing and after pre-processing the data.

* TF-IDF and Bag of words both performed equally well in performance metrics like Recall and F1-score.

* Random Forest performed quite well when compared to Multinomial Naive Bayes.


